In [6]:
import yaml
import os
import numpy as np
import copy

In [7]:
base_config={
                   #'model_name': ,
                   #'model_path': ,
                   #'device': ,
                   'class_groups': [[2*i,2*i+1] for i in range(5)],
                   'dataset_name':'MNIST' ,
                   #'dataset_type':,
                   #'data_root':,
                   #'xpl_root':,
                   #'coef_root':,
                   #'save_dir': ,
                   'validation_size': 2000,
}


In [8]:
# NEED TO UPDATE LOCAL CONFIG GENERATOR FOR PERSONAL LOCAL COMPUTER
def create_config_local(config, config_name):
    if config['dataset_type'] == 'stdk':
        dstype_file = 'std'
    elif config['dataset_type'] == 'groupk':
        dstype_file = 'group'
    elif config['dataset_type'] == 'switched':
        dstype_file = 'switched_one_file'
    elif config["dataset_type"] in ["add_batch_in", "add_batch_in_neg", "leave_out", "only_batch", "lds", "labelflip"]:
        dstype_file = 'std'
    else:
        dstype_file = config['dataset_type']
    config['data_root'] = 'C:/Users/weckbecker/DualView-wip/src/datasets/'
    config['xpl_root'] = f"C:/Users/weckbecker/DualView-wip/explanations/{config['dataset_name']}/{dstype_file}/{config['model_name']}_{dstype_file}/{config['xai_method']}"
    if config['xai_method'] in ['mcsvm', 'representer']:
        base_config['coef_root']=base_config['xpl_root']
    elif 'coef_root' in base_config.keys():
        base_config.pop('coef_root')
    config['device'] = 'cuda'
    config['model_path']=f"C:/Users/weckbecker/DualView-wip/checkpoints/{config['dataset_name']}/{dstype_file}/{config['model_name']}_{dstype_file}/{config['dataset_name']}_{config['model_name']}"
    config['save_dir'] = 'C:/Users/weckbecker/DualView-wip/test_output'
    path = f"local/evaluate/{config['dataset_name']}"
    os.makedirs(path, exist_ok=True)
    
    with open(f"{path}/{config_name}.yaml", 'w') as outfile:
        yaml.dump(config, outfile, default_flow_style=False)
        
def create_config_cluster(config, config_name):
    if config['dataset_type'] == 'stdk':
        dstype_file = 'std'
    elif config['dataset_type'] == 'groupk':
        dstype_file = 'group'
    elif config['dataset_type'] == 'switched':
        dstype_file = 'switched_one_file'
    elif config["dataset_type"] in ["add_batch_in", "add_batch_in_neg", "leave_out", "only_batch", "lds", "labelflip"]:
        dstype_file = 'std'
    else:
        dstype_file = config['dataset_type']
    config['device']='cuda'
    config['xpl_root'] = f'/mnt/explanations/{config['dataset_name']}/{dstype_file}/{config['model_name']}_{dstype_file}/{config['xai_method']}'
    if config['xai_method'] in ['dualview', 'representer']:
        base_config['coef_root']=base_config['xpl_root']
    elif 'coef_root' in base_config.keys():
        base_config.pop('coef_root')
    config['data_root']='/mnt/dataset'
    config['model_path']=f"/mnt/checkpoints/{config['dataset_name']}/{dstype_file}/{config['model_name']}_{dstype_file}/{config['dataset_name']}_{config['model_name']}"
    config['save_dir'] = '/mnt/outputs/'
    
    path = f"cluster/evaluate/{config['dataset_name']}"
    os.makedirs(path, exist_ok=True)
    
    with open(f"{path}/{config_name}.yaml", 'w') as outfile:
        yaml.dump(config, outfile, default_flow_style=False)

In [9]:
model_names=['basic_conv', 'homo_conv']#,'basic_fc', 'homo_fc']
dataset_types=['std', 'group', 'corrupt', 'mark',
               'stdk', 'groupk', 'switched',
               'add_batch_in', "add_batch_in_neg", "leave_out", "only_batch", "lds", "labelflip"]
xai_methods=['dualview_0.001', 'representer', 'influence', 'rp_similarity', 'similarity']

In [10]:
for name in model_names:
    base_config['model_name']=name       
    for dstype in dataset_types:
        if dstype=='group':
            base_config['num_classes']=len(base_config['class_groups'])
        else:
            base_config['num_classes']=10
        base_config['dataset_type']=dstype
        for xai_method in xai_methods:
            base_config['xai_method']=xai_method
            create_config_cluster(base_config, f"{name}_{dstype}_{xai_method}")
            create_config_local(base_config, f"{name}_{dstype}_{xai_method}")